In [1]:
import pandas as pd
import numpy as np
from pathlib import Path
from functions import (load_month, load_month_sa, load_month_pen, mapd_clean_merge)
%pip install xlrd>=2.0.1
!pip install openpyxl

# Settings
monthlist = [f"{m:02d}" for m in range(1, 3)]  # "01", "02"
y = 2017

Note: you may need to restart the kernel to use updated packages.

[notice] A new release of pip is available: 24.2 -> 26.0.1
[notice] To update, run: pip install --upgrade pip


# Plan (enrollment and contract) data

In [2]:
# Load and combine monthly data
plan_data = pd.concat([load_month(m, y) for m in monthlist], ignore_index=True)

# Sort data
plan_data = plan_data.sort_values(['contractid', 'planid', 'state', 'county', 'month'])

# Fill fips by state/county groups
plan_data['fips'] = plan_data.groupby(['state', 'county'])['fips'].transform(
    lambda x: x.ffill().bfill()
)

# Fill plan-level attributes
plan_cols = ['plan_type', 'partd', 'snp', 'eghp', 'plan_name']
for col in plan_cols:
    plan_data[col] = plan_data.groupby(['contractid', 'planid'])[col].transform(
        lambda x: x.ffill().bfill()
    )

# Fill contract-level attributes
contract_cols = ['org_type', 'org_name', 'org_marketing_name', 'parent_org']
for col in contract_cols:
    plan_data[col] = plan_data.groupby(['contractid'])[col].transform(
        lambda x: x.ffill().bfill()
    )

# Sort for aggregation
plan_data = plan_data.sort_values(['contractid', 'planid', 'fips', 'year', 'month'])

# Aggregate to yearly level
def agg_plan_year(group):
    nonmiss = group['enrollment'].notna()
    n = nonmiss.sum()
    enroll_vals = group.loc[nonmiss, 'enrollment']
    
    return pd.Series({
        'n_nonmiss': n,
        'avg_enrollment': enroll_vals.mean() if n > 0 else np.nan,
        'first_enrollment': enroll_vals.iloc[0] if n > 0 else np.nan,
        'last_enrollment': enroll_vals.iloc[-1] if n > 0 else np.nan,
        'state': group['state'].iloc[-1],
        'county': group['county'].iloc[-1],
        'org_type': group['org_type'].iloc[-1],
        'plan_type': group['plan_type'].iloc[-1],
        'partd': group['partd'].iloc[-1],
        'snp': group['snp'].iloc[-1],
        'eghp': group['eghp'].iloc[-1],
        'org_name': group['org_name'].iloc[-1],
        'org_marketing_name': group['org_marketing_name'].iloc[-1],
        'plan_name': group['plan_name'].iloc[-1],
        'parent_org': group['parent_org'].iloc[-1],
        'contract_date': group['contract_date'].iloc[-1]
    })

plan_data_2017 = plan_data.groupby(['contractid', 'planid', 'fips', 'year']).apply(agg_plan_year).reset_index()
print(f"Plan data shape: {plan_data_2017.shape}")

/tmp/ipykernel_4132821/2974104116.py:16: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  lambda x: x.ffill().bfill()


Plan data shape: (2116083, 20)


/tmp/ipykernel_4132821/2974104116.py:54: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  plan_data_2017 = plan_data.groupby(['contractid', 'planid', 'fips', 'year']).apply(agg_plan_year).reset_index()


# Service area data

In [3]:
import sys
sys.executable

# Load and combine monthly service area data
service_year = pd.concat([load_month_sa(m, y) for m in monthlist], ignore_index=True)

# Sort for stable fills
service_year = service_year.sort_values(['contractid', 'fips', 'state', 'county', 'month'])

# Fill fips by state/county groups
service_year['fips'] = service_year.groupby(['state', 'county'])['fips'].transform(
    lambda x: x.ffill().bfill()
)

# Fill contract-level attributes
contract_cols_sa = ['plan_type', 'partial', 'eghp', 'org_type', 'org_name']
for col in contract_cols_sa:
    service_year[col] = service_year.groupby(['contractid'])[col].transform(
        lambda x: x.ffill().bfill()
    )

# Collapse to yearly: one row per contract x county (fips) x year
service_year = service_year.sort_values(['contractid', 'fips', 'year', 'month'])

service_data_2017 = service_year.groupby(['contractid', 'fips', 'year']).agg({
    'state': 'last',
    'county': 'last',
    'org_name': 'last',
    'org_type': 'last',
    'plan_type': 'last',
    'partial': 'last',
    'eghp': 'last',
    'ssa': 'last',
    'notes': 'last'
}).reset_index()

print(f"Service area data shape: {service_data_2017.shape}")

/tmp/ipykernel_4132821/1475410916.py:19: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  lambda x: x.ffill().bfill()
/tmp/ipykernel_4132821/1475410916.py:19: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  lambda x: x.ffill().bfill()


Service area data shape: (320050, 12)


# Plan characteristics (premium) data

In [7]:
MA_COLUMNS_2017 = [
    "state","county","org_name","plan_name","plan_type","premium","partd_deductible",
    "drug_type","gap_coverage","drug_type_detail","contractid",
    "planid","segmentid","moop","star_rating"
]

MA_DTYPES_2017 = {
    "state": "string",
    "county": "string",
    "org_name": "string",
    "plan_name": "string",
    "plan_type": "string",
    "premium": "string",  
    "partd_deductible": "string",
    "drug_type": "string",
    "gap_coverage": "string",
    "drug_type_detail": "string",
    "contractid": "string",
    "planid": "string", 
    "segmentid": "string",   
    "moop": "string",
    "star_rating": "string",
}

MAPD_COLUMNS_2017 = [
    "state","county","org_name","plan_name","contractid","planid","segmentid",
    "org_type","plan_type","snp","snp_type","benefit_type","below_benchmark",
    "national_pdp","premium_partc",
    "premium_partd_basic","premium_partd_supp","premium_partd_total",
    "partd_assist_full","partd_assist_75","partd_assist_50","partd_assist_25",
    "partd_deductible","deductible_exclusions","increase_coverage_limit",
    "gap_coverage"
]

MAPD_DTYPES_2017 = {
    "state": "string",
    "county": "string",
    "org_name": "string",
    "plan_name": "string",
    "contractid": "string",
    "org_type": "string",
    "plan_type": "string",
    "snp": "string",
    "snp_type": "string",
    "benefit_type": "string",
    "below_benchmark": "string",
    "national_pdp": "string",
    "deductible_exclusions": "string",
    "gap_coverage": "string",
    "planid": "string",
    "segmentid": "string",
    "premium_partc": "string",
    "premium_partd_basic": "string",
    "premium_partd_supp": "string",
    "premium_partd_total": "string",
    "partd_assist_full": "string",
    "partd_assist_75": "string",
    "partd_assist_50": "string",
    "partd_assist_25": "string",
    "partd_deductible": "string",
    "increase_coverage_limit": "string",
}

def read_ma_2017_csv(path: str | Path) -> pd.DataFrame:
    return pd.read_csv(
        path,
        skiprows=6,   
        header=None,
        names=MA_COLUMNS_2017,
        dtype=MA_DTYPES_2017,
        low_memory=False,
        encoding="latin1",
    )

def read_mapd_2017_xls(path: str | Path, sheet: str, nrows: int) -> pd.DataFrame:
    return pd.read_excel(
        path,
        engine="xlrd",           
        sheet_name=sheet,
        skiprows=4,
        nrows=nrows,
        header=None,
        names=MAPD_COLUMNS_2017,
        dtype=MAPD_DTYPES_2017,
    )


def load_landscape_2017_inputs(y: int) -> tuple[pd.DataFrame, pd.DataFrame]:
    ma_path_a = Path("../../ma-data/ma/landscape/Extracted Data") / "2017LandscapeSource file MA_AtoM 10182016.csv"
    ma_data_a = read_ma_2017_csv(ma_path_a)

    ma_path_b = Path("../../ma-data/ma/landscape/Extracted Data") / "2017LandscapeSource file MA_NtoW 10182016.csv"
    ma_data_b = read_ma_2017_csv(ma_path_b)

    ma_data = pd.concat([ma_data_a, ma_data_b], ignore_index=True) 

    mapd_path = (
        Path("../../ma-data/ma/landscape/Extracted Data/PartCD/2017")
        / "Medicare Part D 2017 Plan Report 10182016.xls"
    )

    mapd_data_a = read_mapd_2017_xls(mapd_path, sheet="Alabama to Montana", nrows=17046)
    mapd_data_b = read_mapd_2017_xls(mapd_path, sheet="Nebraska to Wyoming", nrows=20396)

    mapd_data = pd.concat([mapd_data_a, mapd_data_b], ignore_index=True)  # R: rbind()

    return ma_data, mapd_data

#def mapd_clean_merge(ma_data: pd.DataFrame, mapd_data: pd.DataFrame, y: int) -> pd.DataFrame:
    #raise NotImplementedError("Translate mapd.clean.merge() from R to Python and plug it in here.")

# Example 
y = 2017
ma_data, mapd_data = load_landscape_2017_inputs(y)
final_landscape = mapd_clean_merge(ma_data=ma_data, mapd_data=mapd_data, y=y)


# Penetration data

In [8]:
# Load and combine monthly penetration data
ma_penetration = pd.concat([load_month_pen(m, y) for m in monthlist], ignore_index=True)

# Sort and fill fips
ma_penetration = ma_penetration.sort_values(['state', 'county', 'month'])
ma_penetration['fips'] = ma_penetration.groupby(['state', 'county'])['fips'].transform(
    lambda x: x.ffill().bfill()
)

# Collapse to yearly
def agg_penetration(group):
    n_elig = group['eligibles'].notna().sum()
    n_enrol = group['enrolled'].notna().sum()
    elig_vals = group['eligibles'].dropna()
    enrol_vals = group['enrolled'].dropna()
    
    return pd.Series({
        'n_elig': n_elig,
        'n_enrol': n_enrol,
        'avg_eligibles': elig_vals.mean() if n_elig > 0 else np.nan,
        'first_eligibles': elig_vals.iloc[0] if n_elig > 0 else np.nan,
        'last_eligibles': elig_vals.iloc[-1] if n_elig > 0 else np.nan,
        'avg_enrolled': enrol_vals.mean() if n_enrol > 0 else np.nan,
        'sd_enrolled': enrol_vals.std() if n_enrol > 1 else np.nan,
        'min_enrolled': enrol_vals.min() if n_enrol > 0 else np.nan,
        'max_enrolled': enrol_vals.max() if n_enrol > 0 else np.nan,
        'first_enrolled': enrol_vals.iloc[0] if n_enrol > 0 else np.nan,
        'last_enrolled': enrol_vals.iloc[-1] if n_enrol > 0 else np.nan,
        'ssa': group['ssa'].iloc[-1]
    })

pen_2017 = ma_penetration.groupby(['fips', 'state', 'county', 'year']).apply(agg_penetration).reset_index()
print(f"Penetration data shape: {pen_2017.shape}")

Penetration data shape: (3276, 16)


/tmp/ipykernel_4132821/481918829.py:32: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  pen_2017 = ma_penetration.groupby(['fips', 'state', 'county', 'year']).apply(agg_penetration).reset_index()


# Rebate data

In [9]:
import sys
!{sys.executable} -m pip install openpyxl

Defaulting to user installation because normal site-packages is not writeable


In [10]:
import sys
sys.executable

# Part C rebate data
ma_path_a = "../../ma-data/ma/cms-payment/2017/2017PartCPlanLevel.xlsx"
risk_rebate_a = pd.read_excel(ma_path_a, skiprows=3, nrows=2824,
                               names=["contractid", "planid", "contract_name", "plan_type",
                                      "riskscore_partc", "payment_partc", "rebate_partc"])

# Clean Part C data
for col in ['riskscore_partc', 'payment_partc', 'rebate_partc']:
    risk_rebate_a[col] = pd.to_numeric(risk_rebate_a[col].astype(str).str.replace(r'[^\d.-]', '', regex=True), errors='coerce')
risk_rebate_a['planid'] = pd.to_numeric(risk_rebate_a['planid'], errors='coerce')
risk_rebate_a['year'] = 2017

# Part D rebate data
ma_path_b = "../../ma-data/ma/cms-payment/2017/2017PartDPlans.xlsx"
risk_rebate_b = pd.read_excel(ma_path_b, skiprows=3, nrows=3898,
                               names=["contractid", "planid", "contract_name", "plan_type",
                                      "directsubsidy_partd", "riskscore_partd", "reinsurance_partd",
                                      "costsharing_partd"])

# Clean Part D data
for col in ['directsubsidy_partd', 'reinsurance_partd', 'costsharing_partd']:
    risk_rebate_b[col] = pd.to_numeric(risk_rebate_b[col].astype(str).str.replace(r'[^\d.-]', '', regex=True), errors='coerce')
risk_rebate_b['payment_partd'] = (risk_rebate_b['directsubsidy_partd'] + 
                                   risk_rebate_b['reinsurance_partd'] + 
                                   risk_rebate_b['costsharing_partd'])
risk_rebate_b['planid'] = pd.to_numeric(risk_rebate_b['planid'], errors='coerce')

# Select relevant columns for Part D
risk_rebate_b = risk_rebate_b[['contractid', 'planid', 'payment_partd',
                               'directsubsidy_partd', 'reinsurance_partd', 'costsharing_partd',
                               'riskscore_partd']]

# Merge Part C and Part D
rebate_2017 = risk_rebate_a.merge(risk_rebate_b, on=['contractid', 'planid'], how='left')
print(f"Rebate data shape: {rebate_2017.shape}")

Rebate data shape: (2809, 13)


# FFS costs data

In [11]:
# FFS costs data
ffs_path = "../../ma-data/ffs-costs/Extracted Data/Aged Only/aged14.csv"
ffs_col_names = ["ssa", "state", "county_name", "parta_enroll",
                 "parta_reimb", "parta_percap", "parta_reimb_unadj",
                 "parta_percap_unadj", "parta_ime", "parta_dsh",
                 "parta_gme", "partb_enroll",
                 "partb_reimb", "partb_percap",
                 "mean_risk"]

ffs_data = pd.read_csv(ffs_path, skiprows=2, names=ffs_col_names, na_values=['*', '.'],
                       usecols=range(15))

# Select and clean
ffscosts_2017 = ffs_data[['ssa', 'state', 'county_name', 'parta_enroll', 'parta_reimb',
                          'partb_enroll', 'partb_reimb', 'mean_risk']].copy()
ffscosts_2017['year'] = 2017
ffscosts_2017['ssa'] = pd.to_numeric(ffscosts_2017['ssa'], errors='coerce')

for col in ['parta_enroll', 'parta_reimb', 'partb_enroll', 'partb_reimb', 'mean_risk']:
    ffscosts_2017[col] = pd.to_numeric(ffscosts_2017[col].astype(str).str.replace(r'[^\d.-]', '', regex=True), errors='coerce')

print(f"FFS costs data shape: {ffscosts_2017.shape}")

FFS costs data shape: (3225, 9)


# Merge data

In [12]:
# Start with plan data, inner join with service area
ma_2017 = plan_data_2017.merge(
    service_data_2017[['contractid', 'fips']],
    on=['contractid', 'fips'],
    how='inner'
)

# Filter out territories and certain plan types
excluded_states = ['VI', 'PR', 'MP', 'GU', 'AS', '']
ma_2017 = ma_2017[
    (~ma_2017['state'].isin(excluded_states)) &
    (ma_2017['snp'] == 'No') &
    ((ma_2017['planid'] < 800) | (ma_2017['planid'] >= 900)) &
    (ma_2017['planid'].notna()) &
    (ma_2017['fips'].notna())
]

# Prepare penetration data for join
pen_2017_join = pen_2017.copy()
pen_2017_join = pen_2017_join.rename(columns={'state': 'state_long', 'county': 'county_long'})
pen_2017_join['state_long'] = pen_2017_join['state_long'].str.lower()

# Keep only unique fips entries
pen_2017_join['ncount'] = pen_2017_join.groupby('fips')['fips'].transform('count')
pen_2017_join = pen_2017_join[pen_2017_join['ncount'] == 1].drop(columns=['ncount'])

# Join penetration data
ma_2017 = ma_2017.merge(pen_2017_join, on='fips', how='left', suffixes=('', '_pen'))

# Create state name lookup
state_2017 = ma_2017.groupby('state').apply(
    lambda g: g.loc[g['state_long'].notna(), 'state_long'].iloc[-1] if g['state_long'].notna().any() else None
).reset_index()
state_2017.columns = ['state', 'state_name']

# Join state names
full_2017 = ma_2017.merge(state_2017, on='state', how='left')

# Prepare landscape data for join
landscape_2017_join = final_landscape.copy()
landscape_2017_join['state'] = landscape_2017_join['state'].str.lower()

landscape_2017_join['premium_partc'] = (
    landscape_2017_join['premium_partc'].fillna(0)
)
landscape_2017_join['premium'] = (
    landscape_2017_join['premium'].fillna(0)
)
# Join landscape data
full_2017 = full_2017.merge(
    landscape_2017_join,
    left_on=['contractid', 'planid', 'state_name', 'county'],
    right_on=['contractid', 'planid', 'state', 'county'],
    how='left',
    suffixes=('', '_land')
)


# Join rebate data (exclude contract_name and plan_type from rebate)
rebate_2017_join = rebate_2017.drop(columns=['contract_name', 'plan_type'], errors='ignore')
full_2017 = full_2017.merge(rebate_2017_join, on=['contractid', 'planid'], how='left', suffixes=('', '_reb'))

full_2017['rebate_partc'] = (
    full_2017['rebate_partc'].fillna(0)
)

full_2017['premium_partc'] = (
    full_2017['premium_partc'].fillna(0)
)

# Calculate basic_premium and bid
def calc_basic_premium(row):
    if row.get('rebate_partc', 0) > 0:
        return 0
    elif row.get('partd') == 'No' and pd.notna(row.get('premium')) and pd.isna(row.get('premium_partc')):
        return row.get('premium')
    else:
        return row.get('premium_partc')

def calc_bid(row):
    rebate = row.get('rebate_partc', 0) or 0
    basic_premium = row.get('basic_premium', 0) or 0
    payment_partc = row.get('payment_partc')
    riskscore_partc = row.get('riskscore_partc')
    
    if pd.isna(payment_partc) or pd.isna(riskscore_partc) or riskscore_partc == 0:
        return np.nan
    elif rebate == 0 and basic_premium > 0:
        return (payment_partc + basic_premium) / riskscore_partc
    elif rebate > 0 or basic_premium == 0:
        return payment_partc / riskscore_partc
    else:
        return np.nan

full_2017['basic_premium'] = full_2017.apply(calc_basic_premium, axis=1)
full_2017['bid'] = full_2017.apply(calc_bid, axis=1)

print(f"Final merged data shape: {full_2017.shape}")

/tmp/ipykernel_4132821/479022999.py:31: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  state_2017 = ma_2017.groupby('state').apply(


Final merged data shape: (54154, 55)


In [13]:
full_2017.head()

,contractid,planid,fips,year,n_nonmiss,avg_enrollment,first_enrollment,last_enrollment,state,county,...,payment_partc,rebate_partc,year_reb,payment_partd,directsubsidy_partd,reinsurance_partd,costsharing_partd,riskscore_partd,basic_premium,bid
0,H0022,1.0,39023.0,2017,2,563.0,554.0,572.0,OH,Clark,...,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN
1,H0022,1.0,39035.0,2017,2,3385.0,3321.0,3449.0,OH,Cuyahoga,...,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN
2,H0022,1.0,39051.0,2017,2,112.0,107.0,117.0,OH,Fulton,...,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN
3,H0022,1.0,39055.0,2017,2,60.0,58.0,62.0,OH,Geauga,...,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN
4,H0022,1.0,39057.0,2017,2,512.5,500.0,525.0,OH,Greene,...,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN


In [14]:
# Save to CSV
output_path = "../data/output/data-2017.csv"
Path(output_path).parent.mkdir(parents=True, exist_ok=True)
full_2017.to_csv(output_path, index=False)
print(f"Data saved to {output_path}")

Data saved to ../data/output/data-2017.csv
